In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta

In [ ]:
#####DB Config
host= "DevEdgeV32"
port=27017

In [ ]:
start= datetime.strptime("2022-11-23T11:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") 
end= datetime.strptime("2022-11-24T14:24:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") 

In [ ]:
time_shift= +1
output_variable= 'welle_z'
prediction_variable= 'prediction'
selected_Columns= ['t_bett','t_motor', 't_spindle','M8','M7', 'M121' , 'M127' , output_variable, prediction_variable]


In [ ]:
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db.modelLogs.find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
for event in event_list:
    if event['date']< start or event['date'] > end:
        continue
    i+=1
    record_list= event['content']
    keys= None
    for record in record_list:
        # if not record['given2model']:
        #     continue
        keys= record['raw_data'].keys()
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
        #print ('before',record)
        record['raw_data']['date']=[record['date'] + timedelta(hours= time_shift)]
        record['raw_data'][output_variable]=[record[output_variable]]
        record['raw_data'][prediction_variable]=[record[prediction_variable]]
        #record['raw_data']['DRZ2'][0]= float(record['raw_data']['DRZ2'][0])
        # record['raw_data']['M8']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M121']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M127']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M7']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0] - record['raw_data']['t_bett'][0], decimals=2)
        # record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]) - record['raw_data']['t_bett'][0], decimals= 2)
        record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0], decimals=2)
        record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]), decimals= 2)
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
signals['t_motor']= signals['t_motor'].apply(np.float32)
signals['prediction_abs_error']= signals[prediction_variable] -signals[output_variable]
signals['prediction_abs_error']= 1000 * signals['prediction_abs_error'].abs()
signals.reset_index(inplace= True)
output=  signals[output_variable]
machine_prediction= signals[prediction_variable]
rough_signals= signals[selected_Columns].to_numpy()
print(rough_signals.shape)
#signals.rename(columns={'t_spindle': 't_spindel', 'DRZ': 'drhz'}, inplace= True)
print (signals.head(3))



In [ ]:
## Comparasion between two sources:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['t_motor', 't_spindle','DRZ2','M8','M7', 'M121' , 'M127' , prediction_variable]
#selected_Columns= ['t_bett','t_motor', 't_spindle','DRZ','DRZ2', prediction_variable]
selected_Columns.append('prediction_abs_error')
print (selected_Columns)
y_axis_names= selected_Columns
fig= make_subplots(rows=len(selected_Columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_Columns, vertical_spacing=0.02)
for i in range(len(selected_Columns)):
    #fig.add_trace(go.Line(x= signals['date'],y= signals[selected_Columns[i]], name=selected_Columns[i]), row= i+1, col= 1)# .rolling(window= 23).mean()
    fig.add_trace(go.Scatter(x= signals['date'],y= signals[selected_Columns[i]], name=selected_Columns[i], mode= scatter_mode), row= i+1, col= 1)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
##Draw the prediciton and the real values of displacement on Welle
#fig.add_trace(go.Line(x= signals['date'],y= signals[output_variable].rolling(window= 23).mean(), name=output_variable), row= len(selected_Columns) - 1, col= 1)
fig.add_trace(go.Scatter(x= signals['date'],y= signals[output_variable], name=output_variable, mode= scatter_mode),  row= len(selected_Columns) - 1, col= 1)
fig.update_yaxes(title_text= output_variable, row= len(selected_Columns)-1, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'Prediction Results')
fig.show()

In [ ]:
def preprocess_data(df: pd.DataFrame, kuelung: list):
    for kuehlungsmittel in kuelung:
        kuehlungsmittel_corrected=  str(kuehlungsmittel + '_corrected')
        df[kuehlungsmittel_corrected] = np.where(
            np.round(df['DRZ2']) >= 300, 1, 0)
    return df

In [ ]:
# ## Comparasion between two sources:
# selected_Columns= ['t_motor', 't_spindle','DRZ','DRZ2','M8','M7', 'M121' , 'M127' , prediction_variable]
# selected_Columns.append('prediction_abs_error')
#signals= preprocess_data(signals, ['M8','M7', 'M121' , 'M127'])
fig= make_subplots(rows=6,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
#'t_motor'
fig.add_trace(go.Line(x= signals['date'],y= (signals['t_motor'] - signals['t_bett']).rolling(window= 23).mean(), name='t_motor'), row= 1, col= 1)
fig.update_yaxes(title_text= 't_motor', row= 1, col= 1)
#'t_spindle'
fig.add_trace(go.Line(x= signals['date'],y= (signals['t_spindle'] - signals['t_bett']).rolling(window= 23).mean(), name='t_spindle'), row= 2, col= 1)
fig.update_yaxes(title_text= 't_spindle', row= 2, col= 1)
# 'DRZ'
fig.add_trace(go.Line(x= signals['date'],y= signals['DRZ'], name='DRZ'), row= 3, col= 1)
fig.update_yaxes(title_text= 'DRZ', row= 3, col= 1)
# 'DRZ2'
fig.add_trace(go.Line(x= signals['date'],y= signals['DRZ2'], name='DRZ2'), row= 4, col= 1)
fig.update_yaxes(title_text= 'DRZ2', row= 4, col= 1)
# # 'M8'
# fig.add_trace(go.Line(x= signals['date'],y= signals['M8'], name='M8'), row= 5, col= 1)
# #fig.add_trace(go.Line(x= signals['date'],y= signals['M8_corrected'], name='M8_corrected'), row= 5, col= 1)
# fig.update_yaxes(title_text= 'M8', row= 5, col= 1)
# fig.add_trace(go.Line(x= signals['date'],y= signals['M8_corrected'], name='M8_corrected'), row= 6, col= 1)
# fig.update_yaxes(title_text= 'M8', row= 6, col= 1)
# # 'M7'
# fig.add_trace(go.Line(x= signals['date'],y= signals['M7'], name='M7'), row= 6, col= 1)
# fig.add_trace(go.Line(x= signals['date'],y= signals['M7_corrected'], name='M7_corrected'), row= 6, col= 1)
# fig.update_yaxes(title_text= 'M7', row= 6, col= 1)
# # 'M121'
# fig.add_trace(go.Line(x= signals['date'],y= signals['M121'], name='M121'), row= 7, col= 1)
# fig.add_trace(go.Line(x= signals['date'],y= signals['M121_corrected'], name='M121_corrected'), row= 7, col= 1)
# fig.update_yaxes(title_text= 'M121', row= 7, col= 1)
# # 'M127'
# fig.add_trace(go.Line(x= signals['date'],y= signals['M127'], name='M127'), row= 8, col= 1)
# fig.add_trace(go.Line(x= signals['date'],y= signals['M127_corrected'], name='M127_corrected'), row= 8, col= 1)
# fig.update_yaxes(title_text= 'M127', row= 8, col= 1)
# prediction_variable
fig.add_trace(go.Line(x= signals['date'],y= signals[prediction_variable], name=prediction_variable), row= 5, col= 1)
fig.update_yaxes(title_text= prediction_variable, row= 5, col= 1)
##Draw the prediciton and the real values of displacement on Welle
fig.add_trace(go.Line(x= signals['date'],y= signals[output_variable], name=output_variable), row= 5, col= 1)
fig.update_yaxes(title_text= output_variable, row= 5, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'Prediction Results')
fig.show()

In [ ]:
## Show the input of the model from db
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db.modelLogs.find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
last_prediction= -500
### Go through events in DB
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
        if 'given2model' in record:
            if not record['given2model']:
                continue
        elif record[prediction_variable] == last_prediction:
            continue
        else:
            last_prediction= record[prediction_variable]
        keys= record['raw_data'].keys()
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
        #print ('before',record)
        record['raw_data']['date']=[record['date'] + timedelta(hours= +2)]
        record['raw_data'][output_variable]=[record[output_variable]]
        record['raw_data'][prediction_variable]=[record[prediction_variable]]
        record['raw_data']['DRZ'][0]= float(record['raw_data']['DRZ'][0])
        # record['raw_data']['M8']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M121']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M127']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M7']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0] - record['raw_data']['t_bett'][0], decimals=2)
        # record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]) - record['raw_data']['t_bett'][0], decimals= 2)
        record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0], decimals=2)
        record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]), decimals= 2)
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
signals['t_motor']= signals['t_motor'].apply(np.float32)
signals['prediction_abs_error']= signals[prediction_variable] -signals[output_variable]
signals['prediction_abs_error']= 1000 * signals['prediction_abs_error'].abs()
signals.reset_index(inplace= True)
output=  signals[output_variable]
machine_prediction= signals[prediction_variable]
rough_signals= signals[selected_Columns].to_numpy()
print(rough_signals.shape)
#signals.rename(columns={'t_spindle': 't_spindel', 'DRZ': 'drhz'}, inplace= True)
print (signals.head(3))


In [ ]:
## Comparasion between two sources:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
selected_Columns= ['t_motor', 't_spindle','DRZ','DRZ2','M8','M7', 'M121' , 'M127' , prediction_variable]
#selected_Columns= ['t_bett','t_motor', 't_spindle','DRZ','DRZ2', prediction_variable]
selected_Columns.append('prediction_abs_error')
print (selected_Columns)
y_axis_names= selected_Columns
fig= make_subplots(rows=len(selected_Columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_Columns, vertical_spacing=0.02)
for i in range(len(selected_Columns)):
    #fig.add_trace(go.Line(x= signals['date'],y= signals[selected_Columns[i]], name=selected_Columns[i]), row= i+1, col= 1)# .rolling(window= 23).mean()
    fig.add_trace(go.Scatter(x= signals['date'],y= signals[selected_Columns[i]], name=selected_Columns[i], mode= scatter_mode), row= i+1, col= 1)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
##Draw the prediciton and the real values of displacement on Welle
#fig.add_trace(go.Line(x= signals['date'],y= signals[output_variable].rolling(window= 23).mean(), name=output_variable), row= len(selected_Columns) - 1, col= 1)
fig.add_trace(go.Scatter(x= signals['date'],y= signals[output_variable], name=output_variable, mode= scatter_mode),  row= len(selected_Columns) - 1, col= 1)
fig.update_yaxes(title_text= output_variable, row= len(selected_Columns)-1, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'Prediction Results')
fig.show()

Regenerate the predictions locally